Test modelu na zbiorze imageNet    

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch.nn as nn
from tqdm import tqdm  # Import tqdm do progress bara

# Sprawdzenie dostępności GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transformacje dla Tiny ImageNet
transform = transforms.Compose([
    transforms.Resize(224),  # Zmiana rozmiaru obrazów na 224x224 (wymagane przez MobileNet)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalizacja, jak w ImageNet
])

# Funkcja do ładowania i testowania danych z różnych folderów (train, val, test)
def test_on_folder(folder_name):
    print(f"\nTesting on folder: {folder_name}")
    
    dataset = ImageFolder(root=f'{data_dir}/{folder_name}', transform=transform)
    loader = DataLoader(dataset, batch_size=32, shuffle=False)
    
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():  # Wyłączenie obliczania gradientów
        progress_bar = tqdm(loader, desc=f"Testing {folder_name}")  # Dodanie progress bara
        for data in progress_bar:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)  # Pobieranie przewidywań
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # Aktualizacja opisu progress bara
            progress_bar.set_postfix(accuracy=f'{100 * correct / total:.2f}%')

    accuracy = 100 * correct / total
    print(f'Test Accuracy on {folder_name}: {accuracy:.2f}%')

# Pobieranie pretrenowanego modelu MobileNetV2
model = torchvision.models.mobilenet_v2(pretrained=True)

# Dostosowanie warstwy klasyfikacyjnej do 200 klas Tiny ImageNet (zamiast 1000 klas ImageNet)
model.classifier[1] = nn.Linear(model.last_channel, 200)
model = model.to(device)

# Ścieżka do zbioru Tiny ImageNet
data_dir = './tiny-imagenet-200/'

# Testowanie na wszystkich folderach: train, val, i test
# folders = ['train', 'val', 'test']
folders = ['val', 'test']
for folder in folders:
    test_on_folder(folder)



Testing on folder: val


Testing val: 100%|██████████| 313/313 [05:22<00:00,  1.03s/it, accuracy=0.00%]


Test Accuracy on val: 0.00%

Testing on folder: test


Testing test:  30%|███       | 94/313 [02:05<05:04,  1.39s/it, accuracy=0.00%]

Test modelu na zbiorze CIFAR

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm  # Importowanie tqdm do paska postępu
import torch.nn as nn

# Sprawdzenie czy mamy dostęp do GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Przygotowanie transformacji dla CIFAR-10 (MobileNet wymaga obrazów 224x224)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Pobieranie zbioru testowego CIFAR-10
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Pobieranie pretrenowanego modelu MobileNetV2
model = torchvision.models.mobilenet_v2(pretrained=True)

# Zmiana warstwy klasyfikującej, aby dopasować model do 10 klas CIFAR-10 (zamiast 1000 z ImageNet)
model.classifier[1] = nn.Linear(model.last_channel, 10)  # CIFAR-10 ma 10 klas
model = model.to(device)

# Funkcja testująca model
def test_model():
    model.eval()  # Ustawienie modelu w tryb ewaluacji
    correct = 0
    total = 0
    
    # Dodanie paska postępu do pętli testowej
    with torch.no_grad():
        for data in tqdm(test_loader, desc="Testing MobileNetV2"):
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)  # Forward pass
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Test Accuracy (bez trenowania na CIFAR-10): {100 * correct / total}%')

# Wywołanie funkcji testowej
test_model()


Douczenie modelu na zbiorze CIFAR

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt

# Sprawdzenie dostępności GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Przygotowanie transformacji dla CIFAR-10
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Pobieranie zbioru danych CIFAR-10
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Pobieranie modelu MobileNetV2 pretrenowanego na ImageNet
model = torchvision.models.mobilenet_v2(pretrained=True)

# Dodanie Dropout po warstwie global_avg_pool (możesz eksperymentować z poziomem dropout)
class MobileNetV2WithDropout(nn.Module):
    def __init__(self, model):
        super(MobileNetV2WithDropout, self).__init__()
        self.features = model.features
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(p=0.3)  # Dodany Dropout
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.3),  # Dropout przed klasyfikacją
            nn.Linear(model.last_channel, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.classifier(x)
        return x

# Przystosowanie modelu do klasyfikacji 10 klas CIFAR-10
model = MobileNetV2WithDropout(model).to(device)

# Funkcje strat i optymalizator
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Przygotowanie zmiennych do przechowywania strat i dokładności
train_losses = []
test_accuracies = []

# Funkcja testująca model
def test_model():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy}%')
    return accuracy

# Trening modelu z progress barem i zapisaniem statystyk
def train_model(epochs=1):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader))
        for i, data in progress_bar:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # Zerowanie gradientów
            optimizer.zero_grad()

            # Forward + backward + optymalizacja
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # Aktualizacja progress bara
            progress_bar.set_description(f'Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / (i + 1):.4f}')
        
        # Zapis strat po każdej epoce
        train_losses.append(running_loss / len(train_loader))
        accuracy = test_model()
        test_accuracies.append(accuracy)

# Uruchomienie treningu na 5 epokach
train_model(epochs=5)

# Rysowanie wykresów strat i dokładności
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(test_accuracies, label='Test Accuracy')
plt.title('Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.show()


Próba ulepszenia mechanizmu doszkalania na zbiorze CIFAR trenowanie tylko klasyfikatora

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt

# Sprawdzenie dostępności GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Przygotowanie transformacji dla CIFAR-10
# Używamy transformacji specyficznej dla CIFAR-10 z odpowiednią normalizacją
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Zmiana rozmiaru na 224x224 (wymagane przez MobileNet)
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),  # Normalizacja CIFAR-10
])

# Pobieranie zbioru danych CIFAR-10 (trening i test)
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Pobieranie modelu MobileNetV2 pretrenowanego na ImageNet
model = torchvision.models.mobilenet_v2(pretrained=True)

# Dodanie Dropout po warstwie global_avg_pool i zamrożenie wcześniejszych warstw
class MobileNetV2WithDropout(nn.Module):
    def __init__(self, model):
        super(MobileNetV2WithDropout, self).__init__()
        # Zamrażanie warstw feature extraction (wymaga, aby ich wagi nie były trenowane)
        for param in model.features.parameters():
            param.requires_grad = False

        # Zostawiamy feature extraction tak jak w oryginalnym MobileNet
        self.features = model.features

        # Pooling i Dropout
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(p=0.3)  # Dodanie Dropout z prawdopodobieństwem 30%

        # Klasyfikacja (10 klas dla CIFAR-10)
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.3),  # Dropout przed warstwą klasyfikującą
            nn.Linear(model.last_channel, 10)  # Dostosowanie do 10 klas CIFAR-10
        )

    def forward(self, x):
        x = self.features(x)  # Przepuszczenie przez warstwy konwolucyjne
        x = self.avgpool(x)  # Zastosowanie pooling
        x = x.view(x.size(0), -1)  # Rozwinięcie tensoru do płaskiej postaci
        x = self.dropout(x)  # Zastosowanie Dropout
        x = self.classifier(x)  # Przepuszczenie przez warstwę klasyfikującą
        return x

# Przystosowanie modelu do klasyfikacji 10 klas CIFAR-10
model = MobileNetV2WithDropout(model).to(device)

# Definiowanie funkcji strat i optymalizatora
criterion = nn.CrossEntropyLoss()  # Używamy funkcji CrossEntropyLoss do klasyfikacji
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
# Używamy optymalizatora Adam, trenowane będą tylko te parametry, które nie są zamrożone (czyli klasyfikator)

# Przygotowanie zmiennych do przechowywania strat i dokładności
train_losses = []
test_accuracies = []

# Funkcja testująca model
def test_model():
    model.eval()  # Ustawienie modelu w tryb ewaluacji
    correct = 0
    total = 0
    with torch.no_grad():  # Wyłączenie gradientów podczas testowania (niepotrzebne w trybie ewaluacji)
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)  # Pobieranie klasy z największym wynikiem
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  # Sumowanie poprawnych predykcji

    accuracy = 100 * correct / total  # Obliczenie dokładności
    print(f'Test Accuracy: {accuracy:.2f}%')
    return accuracy

# Funkcja do treningu modelu z progress barem i zapisaniem statystyk
def train_model(epochs=1):
    for epoch in range(epochs):
        model.train()  # Ustawienie modelu w tryb trenowania
        running_loss = 0.0
        progress_bar = tqdm(enumerate(train_loader), total=len(train_loader))  # Progress bar
        for i, data in progress_bar:
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # Zerowanie gradientów
            optimizer.zero_grad()

            # Forward + backward + optymalizacja
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()  # Obliczanie gradientów
            optimizer.step()  # Aktualizacja wag

            running_loss += loss.item()

            # Aktualizacja progress bara
            progress_bar.set_description(f'Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / (i + 1):.4f}')
        
        # Zapis strat po każdej epoce
        train_losses.append(running_loss / len(train_loader))
        accuracy = test_model()  # Testowanie modelu po każdej epoce
        test_accuracies.append(accuracy)

# Uruchomienie treningu na 5 epokach
train_model(epochs=3)

# Rysowanie wykresów strat i dokładności
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.title('Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(test_accuracies, label='Test Accuracy')
plt.title('Test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.legend()

plt.show()


Test modelu ResNet 50 z Transform Learning

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

# Sprawdzenie czy mamy dostęp do GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Przygotowanie transformacji dla Tiny ImageNet
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Ładowanie zbioru walidacyjnego Tiny ImageNet
data_dir = './tiny-imagenet-200/'
train_dataset = torchvision.datasets.ImageFolder(root=f'{data_dir}/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = torchvision.datasets.ImageFolder(root=f'{data_dir}/val', transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Pobieranie pretrenowanego modelu ResNet50 na ImageNet
model = torchvision.models.resnet50(pretrained=True)

# Dostosowanie warstwy klasyfikującej (ResNet50 ma 1000 wyjść, zmieniamy na 200 dla Tiny ImageNet)
model.fc = nn.Linear(model.fc.in_features, 200)  # Tiny ImageNet ma 200 klas
model = model.to(device)

# Funkcje strat i optymalizator
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Trening modelu z transfer learningiem na Tiny ImageNet
def train_model(epochs=5):
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # Zerowanie gradientów
            optimizer.zero_grad()

            # Forward + backward + optymalizacja
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}')

# Testowanie modelu na zbiorze walidacyjnym
def test_model():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in val_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Validation Accuracy: {100 * correct / total:.2f}%')

# Trening i testowanie modelu
train_model(epochs=5)
test_model()
